In [1]:
import os

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Gemma2-9b-It")

In [9]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1084efa60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x108629c90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
llm.invoke("What is the capital of France?").content

'The capital of France is **Paris**.  \n'

## Tools

In [12]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [13]:
api_wrapper = WikipediaAPIWrapper()
tool = WikipediaQueryRun(api_wrapper=api_wrapper, return_direct=True, verbose=True)

In [15]:
print(tool.run("Who is Max Verstappen?"))

Page: Max Verstappen
Summary: Max Emilian Verstappen (Dutch pronunciation: [ˈmɑks fɛrˈstɑpə(n)]; born 30 September 1997) is a Dutch and Belgian racing driver who competes under the Dutch flag in Formula One for Red Bull Racing. Verstappen has won four Formula One World Drivers' Championship titles, which he won consecutively from 2021 to 2024 with Red Bull, and has won 65 Grands Prix across 11 seasons.
Born in Hasselt and raised in Maaseik, Verstappen is the son of Dutch former Formula One driver Jos Verstappen and Belgian former kart racer Sophie Kumpen. After a successful karting career—culminating in his record-breaking 2013 season—Verstappen graduated to junior formulae. Progressing directly to FIA European Formula 3, Verstappen broke several records on his way to third in the championship in his rookie season with Van Amersfoort. Aged 17, Verstappen signed for Toro Rosso in 2015 as part of the Red Bull Junior Team, becoming the youngest driver in Formula One history at the Austral

## Create your custom tool

In [16]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Get the length of a word."""
    return len(word)


# We have to define a docstring for the tool to work properly, without it, it won't show up in the list of tools the agent can use and give error.

In [18]:
get_word_length.invoke("Hello World")

11

In [19]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

In [22]:
add_numbers.invoke({"a": 5, "b": 10})

15

## Concept of Agents

In [23]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [25]:
tools = load_tools(["wikipedia"], llm=llm)

In [30]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


In [31]:
agent.run("Who is the current world champion in Formula 1 racing?")

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_46430/497327816.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who is the current world champion in Formula 1 racing?")




> Entering new AgentExecutor chain...
Thought: I need to look up the current Formula 1 world champion on Wikipedia.
Action: wikipedia
Action Input: Formula 1 World Championship
Observation: Page: 2024 Formula One World Championship
Summary: The 2024 FIA Formula One World Championship was a motor racing championship for Formula One cars and was the 75th running of the Formula One World Championship. It was recognised by the Fédération Internationale de l'Automobile (FIA), the governing body of international motorsport, as the highest class of competition for open-wheel racing cars. The championship was contested over a record twenty-four Grands Prix held around the world.
Drivers and teams competed for the titles of World Drivers' Champion and World Constructors' Champion, respectively. Defending Drivers' Champion Max Verstappen of Red Bull Racing started off the season with seven wins in the opening 10 races, but was pressured by McLaren driver Lando Norris for the rest of the season

'Max Verstappen'

In [34]:
agent.run("What is GPT-2?")



> Entering new AgentExecutor chain...
Thought: I need to look up GPT-2 on Wikipedia to find out what it is.  
Action: wikipedia
Action Input: GPT-2
Observation: Page: GPT-2
Summary: Generative Pre-trained Transformer 2 (GPT-2) is a large language model by OpenAI and the second in their foundational series of GPT models. GPT-2 was pre-trained on a dataset of 8 million web pages. It was partially released in February 2019, followed by full release of the 1.5-billion-parameter model on November 5, 2019.
GPT-2 was created as a "direct scale-up" of GPT-1 with a ten-fold increase in both its parameter count and the size of its training dataset. It is a general-purpose learner and its ability to perform the various tasks was a consequence of its general ability to accurately predict the next item in a sequence, which enabled it to translate texts, answer questions about a topic from a text, summarize passages from a larger text, and generate text output on a level sometimes indistinguishabl

'GPT-2 is a large language model by OpenAI and the second in their foundational series of GPT models. It was pre-trained on a dataset of 8 million web pages and is capable of performing various tasks such as translation, answering questions, summarizing text, and generating text.'

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily basically provides a search engine tool that can be used to fetch real-time information from the web.

In [36]:
search = TavilySearchResults()

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_46430/1244456812.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults()


In [37]:
search.run("Who is the current world champion in Formula 1 racing?")

[{'title': 'F1 World Champions by Year | Full Historical Overview',
  'url': 'https://racingnews365.com/every-world-champion-in-formula-1-history',
  'content': "The first World Championship of Formula One occurred in 1950, where Giuseppe Farina won the first F1 world title ever. A lot of legendary drivers have made it onto the F1 world champion driver list, such as Alain Prost, Ayrton Senna, Michael Schumacher, and many more. Currently, both Lewis Hamilton and Michael Schumacher hold the record for the most F1 world titles ever, with an outstanding total of seven championships each. [...] Currently, Sebastian Vettel is the youngest world champion in F1 history. He won his first F1 world title in 2010 at the age of 23 years and 134 days with the team Red Bull. The German driver is known as one of the most successful drivers in Formula One history and won from 2010 until 2013 with the same team. Vettel finished his last season in F1 at Aston Martin, retiring at the age of 36 at the end 

In [38]:
search.invoke("Who is the current world champion in Formula 1 racing?")

[{'title': 'F1 World Champions by Year | Full Historical Overview',
  'url': 'https://racingnews365.com/every-world-champion-in-formula-1-history',
  'content': "The first World Championship of Formula One occurred in 1950, where Giuseppe Farina won the first F1 world title ever. A lot of legendary drivers have made it onto the F1 world champion driver list, such as Alain Prost, Ayrton Senna, Michael Schumacher, and many more. Currently, both Lewis Hamilton and Michael Schumacher hold the record for the most F1 world titles ever, with an outstanding total of seven championships each. [...] Currently, Sebastian Vettel is the youngest world champion in F1 history. He won his first F1 world title in 2010 at the age of 23 years and 134 days with the team Red Bull. The German driver is known as one of the most successful drivers in Formula One history and won from 2010 until 2013 with the same team. Vettel finished his last season in F1 at Aston Martin, retiring at the age of 36 at the end 

In [39]:
tools = [search]

In [44]:
from langchain import hub

In [45]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [48]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [47]:
from langchain.agents import create_tool_calling_agent

agent1 = create_tool_calling_agent(llm, tools, prompt)

In [49]:
from langchain.agents import AgentExecutor

In [58]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=6,
    early_stopping_method="generate",   # <- if it’s stuck, ask LLM to write a final answer
)

In [52]:
agent_executor.invoke({"input": "Who is John Wick?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Who is John Wick'}`


[{'title': 'John Wick | Branson Universe Wiki - Fandom', 'url': 'https://branson-universe.fandom.com/wiki/John_Wick', 'content': 'John Wick is a fictional character created by Derek Kolstad, and the titular protagonist of the neo-noir action thriller film series _John Wick_, portrayed by Keanu Reeves. John is a legendary hitman who had retired until a gang invades his house, steals his car, and kills the puppy that his late wife Helen had given him. This sets him on a path of revenge, returning him to the criminal underworld and eventually coming into conflict with the international organization of assassins of which he [...] The character and franchise has been described as a "central boogeyman character who can\'t be killed" and a "modern, action-oriented equivalent of \'80s horror franchises", and reportedly has a kill count of 306, exceeding Jason Voorhees and Mich

{'input': 'Who is John Wick?',
 'output': 'John Wick is a fictional character created by Derek Kolstad, and the titular protagonist of the neo-noir action thriller film series _John Wick_, portrayed by Keanu Reeves. John is a legendary hitman who had retired until a gang invades his house, steals his car, and kills the puppy that his late wife Helen had given him. This sets him on a path of revenge, returning him to the criminal underworld and eventually coming into conflict with the international organization of assassins of which he [...]'}